In [ ]:
import torch

In [ ]:
torch.cuda.device(0)

In [ ]:
torch.cuda.get_device_name()

'Tesla T4'

In [ ]:
torch.cuda.is_available()

True

In [ ]:
num_epochs = 5 
batch_size = 100
learning_rate = 0.001

In [ ]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
train_dataset = MNIST(root = './', train=True, download=True, transform=transforms.ToTensor())
test_dataset= MNIST(root = './', train=False, download=True, transform=transforms.ToTensor())

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
import torch.utils.data as Data
train_loader = Data.DataLoader(dataset=train_dataset, 
                               batch_size=batch_size, 
                               shuffle=True)
test_loader = Data.DataLoader(dataset=test_dataset, 
                               batch_size=batch_size, 
                               shuffle=False) # shuffle=False

In [ ]:
import torch.nn as nn

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(stride=2, kernel_size=2)
    )
    self.dense = nn.Sequential(
        nn.Linear(in_features=14*14*128, out_features=1024),
        nn.ReLU(),
        nn.Linear(1024, 10)
    )

  def forward(self, x):
    out = self.conv(x)
    out = out.view(-1, 14*14*128)
    out = self.dense(out)
    return out

In [ ]:
model = Model()
model.cuda()

Model(
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)

In [ ]:
import torch.optim as opt
from torch.autograd import Variable

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = opt.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
  for idx, (images, labels) in enumerate(train_loader):
    images = Variable(images.type(torch.cuda.FloatTensor))
    labels = Variable(labels.type(torch.cuda.LongTensor))

    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_func(outputs, labels)
    loss.backward()
    optimizer.step()

    if (idx+1)%100 == 0:
      print("Epoch: %d, Batch: %d, Loss: %.4f" %(epoch+1, idx+1, loss.data))

Epoch: 1, Batch: 100, Loss: 0.0924
Epoch: 1, Batch: 200, Loss: 0.1911
Epoch: 1, Batch: 300, Loss: 0.0569
Epoch: 1, Batch: 400, Loss: 0.0910
Epoch: 1, Batch: 500, Loss: 0.0543
Epoch: 1, Batch: 600, Loss: 0.0465
Epoch: 2, Batch: 100, Loss: 0.0945
Epoch: 2, Batch: 200, Loss: 0.0473
Epoch: 2, Batch: 300, Loss: 0.0773
Epoch: 2, Batch: 400, Loss: 0.0186
Epoch: 2, Batch: 500, Loss: 0.0053
Epoch: 2, Batch: 600, Loss: 0.0572
Epoch: 3, Batch: 100, Loss: 0.0042
Epoch: 3, Batch: 200, Loss: 0.0085
Epoch: 3, Batch: 300, Loss: 0.0112
Epoch: 3, Batch: 400, Loss: 0.0618
Epoch: 3, Batch: 500, Loss: 0.0114
Epoch: 3, Batch: 600, Loss: 0.0069
Epoch: 4, Batch: 100, Loss: 0.0281
Epoch: 4, Batch: 200, Loss: 0.0139
Epoch: 4, Batch: 300, Loss: 0.0383
Epoch: 4, Batch: 400, Loss: 0.0103
Epoch: 4, Batch: 500, Loss: 0.0003
Epoch: 4, Batch: 600, Loss: 0.0015
Epoch: 5, Batch: 100, Loss: 0.0000
Epoch: 5, Batch: 200, Loss: 0.0449
Epoch: 5, Batch: 300, Loss: 0.0004
Epoch: 5, Batch: 400, Loss: 0.0003
Epoch: 5, Batch: 500

In [ ]:
correct = 0
total = 0
for images, labels in test_loader:
  images = Variable(images.type(torch.cuda.FloatTensor))
  outputs = model(images)
  _, pred = torch.max(outputs.data, 1) # 1=(from) row
  
  correct += (pred == labels.type(torch.cuda.LongTensor)).sum() # 猜對量sum
  total += labels.size(0)

print('Accuracy:%.3f%%' %(100.0 * float(correct)/float(total)))

Accuracy:98.950%
